# Importações

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import plotly.express as px
from sqlalchemy import create_engine
from statsmodels.tsa.seasonal import STL
from statsforecast import StatsForecast
from statsforecast.models import MSTL, AutoARIMA
from utilsforecast.plotting import plot_series
from datetime import date, timedelta

c:\Users\joana\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Conexão

In [2]:
df_daily_filled = pd.read_csv('../df_daily_filled.csv', parse_dates=['detected'], dtype='int32')
df_daily_filled = df_daily_filled.set_index('detected')
df_daily_filled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 943 entries, 2021-08-01 to 2024-02-29
Data columns (total 62 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   direction_2   943 non-null    int32
 1   count_2       943 non-null    int32
 2   direction_3   943 non-null    int32
 3   count_3       943 non-null    int32
 4   direction_4   943 non-null    int32
 5   count_4       943 non-null    int32
 6   direction_5   943 non-null    int32
 7   count_5       943 non-null    int32
 8   direction_6   943 non-null    int32
 9   count_6       943 non-null    int32
 10  direction_7   943 non-null    int32
 11  count_7       943 non-null    int32
 12  direction_8   943 non-null    int32
 13  count_8       943 non-null    int32
 14  direction_9   943 non-null    int32
 15  count_9       943 non-null    int32
 16  direction_10  943 non-null    int32
 17  count_10      943 non-null    int32
 18  direction_11  943 non-null    int32
 19  count_11  

In [3]:
df_loc = pd.read_csv('../all_counter_locations.csv')
df_loc.info()
print()
print(df_daily_filled.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   locationId         31 non-null     int64  
 1   tenantIdentifier   31 non-null     object 
 2   name               31 non-null     object 
 3   cp7                31 non-null     object 
 4   freguesia          31 non-null     object 
 5   directionPositive  0 non-null      float64
 6   directionNegative  0 non-null      float64
 7   latitude           31 non-null     float64
 8   longitude          31 non-null     float64
dtypes: float64(4), int64(1), object(4)
memory usage: 2.3+ KB

            direction_2  count_2  direction_3  count_3  direction_4  count_4  \
detected                                                                       
2021-08-01          219      327          155      323          138      424   
2021-08-02          144      252          194      381          117

In [4]:
#sem variavel exogena
df_daily_filled = df_daily_filled.resample('D').sum()
df_8 = df_daily_filled.filter(['count_8']).reset_index().assign(unique_id = 8)
df_8 = df_8.filter(['unique_id','detected','count_8'])
df_8.columns = ['unique_id','ds','y']
df_8

,unique_id,ds,y
0,8,2021-08-01,106
1,8,2021-08-02,91
2,8,2021-08-03,115
3,8,2021-08-04,104
4,8,2021-08-05,107
...,...,...,...
938,8,2024-02-25,75
939,8,2024-02-26,112
940,8,2024-02-27,137
941,8,2024-02-28,156


In [5]:
def calcular_pascoa(ano):
    """Calcula a data da Páscoa para um dado ano (algoritmo de Meeus/Jones/Butcher)."""
    a = ano % 19
    b = ano // 100
    c = ano % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2 * e + 2 * i - h - k) % 7
    m = (a + 11 * h + 22 * l) // 451
    mes = (h + l - 7 * m + 114) // 31
    dia = ((h + l - 7 * m + 114) % 31) + 1
    return date(ano, mes, dia)

#feriados 
def calcular_feriados_portugal(anos, incluir_lisboa=False):
    feriados = []

    for ano in anos:
        # Feriados
        feriados += [
            date(ano, 1, 1),   # Ano Novo
            date(ano, 4, 25),  # Dia da Liberdade
            date(ano, 5, 1),   # Dia do Trabalhador
            date(ano, 6, 10),  # Dia de Portugal
            date(ano, 8, 15),  # Assunção de Nossa Senhora
            date(ano, 10, 5),  # Implantação da República
            date(ano, 11, 1),  # Dia de Todos os Santos
            date(ano, 12, 1),  # Restauração da Independência
            date(ano, 12, 8),  # Imaculada Conceição
            date(ano, 12, 25), # Natal
            date(ano, 6, 13),  # Santo António
        ]

        # Feriados Domingo de Páscoa
        pascoa = calcular_pascoa(ano)
        feriados += [
            pascoa - timedelta(days=47), # Carnaval
            pascoa - timedelta(days=2),  # Sexta-feira Santa
            pascoa + timedelta(days=1),  # Segunda-feira de Páscoa 
            pascoa + timedelta(days=60), # Corpo de Deus
        ]

    return set(feriados)


In [6]:
anos = df_daily_filled.index.year.unique()
feriados = calcular_feriados_portugal(anos, incluir_lisboa=True)

df_daily_filled['holiday'] = df_daily_filled.index.date
df_daily_filled['holiday'] = df_daily_filled['holiday'].apply(
    lambda x: 1 if x in feriados else 0
)


In [7]:
#fim de semana
df_daily_filled['weekend'] = df_daily_filled.index.weekday.isin([5, 6]).astype(int)
df_daily_filled['exog'] = ((df_daily_filled['holiday'] == 1) | (df_daily_filled['weekend'] == 1)).astype(int)


In [8]:
#com variavel exogena 
df_8_1 = df_daily_filled[['count_8', 'exog']].reset_index().assign(unique_id=8)
df_8_1.columns = ['ds', 'y', 'X', 'unique_id']
df_8_1 = df_8_1[['unique_id', 'ds', 'y', 'X']].query('ds < "2022-08-01"')
df_8_1

,unique_id,ds,y,X
0,8,2021-08-01,106,1
1,8,2021-08-02,91,0
2,8,2021-08-03,115,0
3,8,2021-08-04,104,0
4,8,2021-08-05,107,0
...,...,...,...,...
360,8,2022-07-27,131,0
361,8,2022-07-28,115,0
362,8,2022-07-29,99,0
363,8,2022-07-30,75,1


# MSTL forecast

In [9]:
#model = AutoARIMA(season_length=7)
model = MSTL(season_length=[7], trend_forecaster=AutoARIMA())

sf = StatsForecast(
    models=[model],
    freq='D',
    n_jobs=-1
)

sf.fit(df_8_1)

# Criar o DataFrame de previsão para o próximo dia
future_dates = pd.date_range(start='2024-03-01', periods=1, freq='D')
future_exog = pd.DataFrame({
    'ds': future_dates,
    'X': [0]  
})
future_exog['unique_id'] = 8  

# Realizar a previsão
levels=[99.7]
fcst = sf.forecast(df=df_8_1, X_df=future_exog, h=1, level=levels, fitted=True)

# Exibir as previsões
print(fcst)


   unique_id         ds        MSTL  MSTL-lo-99.7  MSTL-hi-99.7
0          8 2022-08-01  118.129478     49.413067    186.845886


In [10]:
# models = [MSTL(season_length = [7])]
# sf = StatsForecast(
#     models=models, 
#     freq='D',
#     n_jobs=-1,
# )

# horizon = 7
# levels = [99] 

# fcst = sf.forecast(df=df_8_1, h=7, level=[99], fitted=True)
# fcst.head()

In [11]:
insample_forecasts = sf.forecast_fitted_values()
insample_forecasts.head()


,unique_id,ds,y,MSTL,MSTL-lo-99.7,MSTL-hi-99.7
0,8,2021-08-01,106.0,97.394295,28.677883,166.110703
1,8,2021-08-02,91.0,98.473061,29.756651,167.189468
2,8,2021-08-03,115.0,95.154320,26.437908,163.870728
3,8,2021-08-04,104.0,108.334389,39.617977,177.050797
4,8,2021-08-05,107.0,99.840744,31.124329,168.557159


In [12]:
# anomalies = insample_forecasts[~insample_forecasts['y'].between(insample_forecasts['MSTL-lo-99'], insample_forecasts['MSTL-hi-99'])]
# anomalies
anomalies = insample_forecasts[
    ~insample_forecasts['y'].between(
        insample_forecasts.iloc[:,4],
        insample_forecasts.iloc[:,5]
    )
]

anomalies

,unique_id,ds,y,MSTL,MSTL-lo-99.7,MSTL-hi-99.7
146,8,2021-12-25,276.0,91.666786,22.950373,160.383194
147,8,2021-12-26,389.0,228.984833,160.268417,297.701233
148,8,2021-12-27,50.0,174.463547,105.747131,243.179962


In [13]:
plot_series(forecasts_df=insample_forecasts, level=levels, plot_anomalies=True, engine='plotly')

# Função dinamica

In [14]:
def fill_zeros(df_daily_filled, max_weeks_back=5):
    df_filled = df_daily_filled.copy()
    for col in df_daily_filled.columns:
        if col.startswith('count_'):
            zero_dates = df_daily_filled[df_daily_filled[col] == 0].index
            for datetime in zero_dates:
                for weeks_back in range(1, max_weeks_back + 1):
                    prev_week = datetime - pd.Timedelta(weeks=weeks_back)
                    if prev_week in df_daily_filled.index:
                        prev_value = df_daily_filled.loc[prev_week, col]
                        if prev_value != 0:
                            df_filled.loc[datetime, col] = prev_value
                            break
    return df_filled


In [15]:
def aplica_algoritmo(df_daily_filled, contador_id, exog_col='exog',start_date='2021-08-01', end_date='2022-08-01', season_length=[7], n_jobs=-1, datas_corrigidas_previamente=None):
    
    if contador_id is None:
        raise ValueError("É necessário especificar o 'contador_id'.")
    
    if contador_id not in df_daily_filled.columns:
        raise ValueError(f"Contador {contador_id} não encontrado no DataFrame.")
    
    if exog_col not in df_daily_filled.columns:
        raise ValueError(f"Coluna exógena {exog_col} não encontrada no DataFrame.")
    
    if datas_corrigidas_previamente is None:
        datas_corrigidas_previamente = set()
    
    # 1. Filtrar dados históricos até à data limite
    df_hist = df_daily_filled.loc[start_date:end_date].copy()
    
    # Forçar coluna do contador para float para evitar warnings
    df_hist[contador_id] = df_hist[contador_id].astype(float)

    # 2. Substituir zeros com o valor da semana anterior
    df_hist_corr = fill_zeros(df_hist)

    # 3. Preparar dados para StatsForecast
    unique_id = contador_id
    df_model = df_hist_corr[[contador_id, exog_col]].reset_index().assign(unique_id=unique_id)
    df_model.columns = ['ds', 'y', 'X', 'unique_id']
    df_model = df_model[['unique_id', 'ds', 'y', 'X']]

    # 4. Ajustar o modelo
    model = MSTL(season_length=season_length, trend_forecaster=AutoARIMA())
    sf = StatsForecast(models=[model], freq='D', n_jobs=n_jobs)
    sf.fit(df_model)
    
    future_dates = pd.date_range(start=df_model['ds'].max() + pd.Timedelta(days=1), periods=1, freq='D')

    # Criar lista com 0/1 indicando se cada data é feriado
    future_exog_values = [1 if d.date() in feriados else 0 for d in future_dates]

    future_exog = pd.DataFrame({
        'ds': future_dates,
        'X': future_exog_values,
        'unique_id': [contador_id] * len(future_dates)      
    })

    # Realizar a previsão
    levels=[99.7]
    fcst = sf.forecast(df=df_model, X_df=future_exog, h=1, level=levels, fitted=True)

    # 5. Obter previsões in-sample e identificar outliers
    insample_fcst = sf.forecast_fitted_values()
    insample_fcst.columns = ['unique_id', 'ds', 'y', 'fitted', 'lo_99_7', 'hi_99_7']

    # 6. Detetar anomalias
    anomalies = insample_fcst[
        ~insample_fcst['y'].between(insample_fcst['lo_99_7'], insample_fcst['hi_99_7'])
    ].copy()

    novas_datas_corrigidas = set()

    #7. corrigir anomalias
    for _, row in anomalies.iterrows():
        dia = row['ds']
        
        if dia in datas_corrigidas_previamente or dia in novas_datas_corrigidas:
            continue  # já corrigido anteriormente, ignorar

        y = row['y']
        hi = row['hi_99_7']
        lo = row['lo_99_7']
        historico = df_hist_corr.loc[df_hist_corr.index < dia, contador_id]

        if historico.empty:
            continue

        max_hist = historico.max()

        if y > hi:
            novo_valor = min(hi, max_hist)
            print(f"Corrigindo valor alto no dia {dia}: y={y}, hi={hi}, lo={lo}, max_hist={max_hist} -> novo_valor={novo_valor}")
        elif y < lo:
            novo_valor = lo
            print(f"Corrigindo valor baixo no dia {dia}: y={y}, hi={hi}, lo={lo}, max_hist={max_hist} -> novo_valor={novo_valor}")
        else:
            continue

        df_hist_corr.loc[dia, contador_id] = novo_valor
        novas_datas_corrigidas.add(dia)

    # Devolver histórico corrigido, anomalias e datas que foram corrigidas agora
    return df_hist_corr, anomalies, novas_datas_corrigidas


In [44]:
# def get_one_more_week(df_daily_filled, df_hist, weeks_added, max_weeks=2):
#     if weeks_added >= max_weeks:
#         return df_hist, weeks_added, False

#     last_date = df_hist.index.max()
#     next_week_start = last_date + pd.Timedelta(days=1)
#     next_week_end = next_week_start + pd.Timedelta(days=6)
#     df_next_week = df_daily_filled.loc[next_week_start:next_week_end]

#     if df_next_week.empty:
#         return df_hist, weeks_added, False

#     df_hist_updated = pd.concat([df_hist, df_next_week])
#     return df_hist_updated, weeks_added + 1, True


def get_one_more_week(df_daily_filled, df_hist, weeks_added):
    last_date = df_hist.index.max()
    next_week_start = last_date + pd.Timedelta(days=1)
    next_week_end = next_week_start + pd.Timedelta(days=6)

    df_next_week = df_daily_filled.loc[next_week_start:next_week_end]

    if df_next_week.empty:
        return df_hist, weeks_added, False

    df_hist_updated = pd.concat([df_hist, df_next_week])
    return df_hist_updated, weeks_added + 1, True


In [45]:
# # Dicionários para guardar resultados de cada contador <- para todos os contadores
# resultados_corrigidos_por_contador = {}
# anomalias_por_contador = {}

resultados_corrigidos = [] # <- quando for para todos os contadores esta linha e as 2 proximas não são necessarias
todas_anomalias = []
datas_corrigidas_global = set()

contador_id = 'count_8' # <- quando for para todos os contadores tem de ser eliminada
exog_col = 'exog'
start_date = '2021-08-01'
end_date_inicial = '2022-07-31'
#max_semanas = 2
semanas_adicionadas = 0 #quando for para todos os contadores tem de ser eliminada

# for contador_id in colunas_contadores:  <- par todos os contadores
#     if not contador_id.startswith("count_"):
#         continue  # ignora colunas que não são contadores

#     print(f"\n### PROCESSANDO {contador_id} ###")

#     resultados_corrigidos = []
#     todas_anomalias = []
#     datas_corrigidas_global = set()
#     semanas_adicionadas = 0

    # Aplicar primeira vez
df_hist_atual, anomalies_iniciais, datas_corrigidas = aplica_algoritmo(
    df_daily_filled=df_daily_filled,
    contador_id=contador_id,
    exog_col=exog_col,
    start_date=start_date,
    end_date=end_date_inicial,
    datas_corrigidas_previamente=datas_corrigidas_global
)

    # Atualizar listas
datas_corrigidas_global.update(datas_corrigidas)
resultados_corrigidos.append(df_hist_atual.copy())
todas_anomalias.append(anomalies_iniciais)

    # Loop
while True:    
#while semanas_adicionadas < max_semanas:
    df_hist_atual, semanas_adicionadas, conseguiu_adicionar = get_one_more_week(
        df_daily_filled=df_daily_filled,
        df_hist=df_hist_atual,
        weeks_added=semanas_adicionadas
        #max_weeks=max_semanas
    )

    if not conseguiu_adicionar:
        print("Não há mais semanas para adicionar.")
        break

    df_corrigido, novas_anomalias, novas_datas_corrigidas = aplica_algoritmo(
        df_daily_filled=df_hist_atual,
        contador_id=contador_id,
        exog_col=exog_col,
        start_date=start_date,
        end_date=df_hist_atual.index.max().strftime('%Y-%m-%d'),
        datas_corrigidas_previamente=datas_corrigidas_global
    )

    # Filtrar anomalias para ignorar as que já foram corrigidas antes
    novas_anomalias = novas_anomalias[~novas_anomalias['ds'].isin(datas_corrigidas_global)].copy()

    # Atualizar conjunto global das datas corrigidas
    datas_corrigidas_global.update(novas_datas_corrigidas)

    df_hist_atual = df_corrigido.copy()
    resultados_corrigidos.append(df_hist_atual.copy())
    todas_anomalias.append(novas_anomalias)

    print(f"Semana {semanas_adicionadas} processada com {len(novas_anomalias)} novos outliers.")


    # # Guardar resultados <- para todos os contadores
    # resultados_corrigidos_por_contador[contador_id] = resultados_corrigidos
    # anomalias_por_contador[contador_id] = pd.concat(todas_anomalias, ignore_index=True)

Corrigindo valor alto no dia 2021-12-25 00:00:00: y=276.0, hi=160.38319699200875, lo=22.95037362979061, max_hist=177.0 -> novo_valor=160.38319699200875
Corrigindo valor alto no dia 2021-12-26 00:00:00: y=389.0, hi=297.7012478541393, lo=160.26842449192117, max_hist=177.0 -> novo_valor=177.0
Corrigindo valor baixo no dia 2021-12-27 00:00:00: y=50.0, hi=243.17995478686115, lo=105.74713142464303, max_hist=177.0 -> novo_valor=105.74713142464303
Corrigindo valor alto no dia 2021-11-04 00:00:00: y=177.0, hi=162.3913586430818, lo=57.06150217876634, max_hist=141.0 -> novo_valor=141.0
Semana 1 processada com 1 novos outliers.
Corrigindo valor baixo no dia 2022-08-08 00:00:00: y=14.0, hi=174.85335290802914, lo=65.41588301753576, max_hist=177.0 -> novo_valor=65.41588301753576
Semana 2 processada com 1 novos outliers.
Semana 3 processada com 0 novos outliers.
Semana 4 processada com 0 novos outliers.
Semana 5 processada com 0 novos outliers.
Semana 6 processada com 0 novos outliers.
Corrigindo valo

In [46]:
#Juntar todas as anomalias detetadas 
anomalias_totais = pd.concat(todas_anomalias)
anomalias_totais

# Para todos os contadores
# anomalias_totais = pd.concat(
#     [df.assign(contador=contador_id) for contador_id, df in anomalias_por_contador.items() if not df.empty],
#     ignore_index=True
# )

# if anomalias_totais.empty:
#     print("Nenhuma anomalia encontrada.")
# else:
#     print(anomalias_totais[['ds', 'contador', 'y', 'lo_99_7', 'hi_99_7']])



,unique_id,ds,y,fitted,lo_99_7,hi_99_7
146,count_8,2021-12-25,276.0,91.666785,22.950374,160.383197
147,count_8,2021-12-26,389.0,228.984836,160.268424,297.701248
148,count_8,2021-12-27,50.0,174.463543,105.747131,243.179955
95,count_8,2021-11-04,177.0,109.726430,57.061502,162.391359
372,count_8,2022-08-08,14.0,120.134618,65.415883,174.853353
408,count_8,2022-09-13,233.0,165.002138,110.725831,219.278444
435,count_8,2022-10-10,84.0,143.023691,89.011442,197.035941
444,count_8,2022-10-19,90.0,147.041531,92.542082,201.540980
450,count_8,2022-10-25,281.0,187.108394,131.044431,243.172357
523,count_8,2023-01-06,187.0,127.940155,69.524236,186.356074


In [47]:
df_corrigido_final = resultados_corrigidos[-1]
fig = px.line(df_corrigido_final, y=contador_id, 
                title="Contagem corrigida",
                labels={"index": "Data", contador_id: "Contagem"})
fig.update_layout(width=1300, height=350)
fig.show()


# Se eu quiser fazer para todos os contadores

# for contador_id, lista_resultados in resultados_corrigidos_por_contador.items():
#     if not lista_resultados:
#         continue  # pular se a lista estiver vazia

#     df_corrigido_final = lista_resultados[-1]  # último resultado corrigido

#     fig = px.line(
#         df_corrigido_final,
#         y=contador_id,
#         title=f"Contagem corrigida - {contador_id}",
#         labels={"index": "Data", contador_id: "Contagem"}
#     )
#     fig.update_layout(width=1300, height=350)
#     fig.show()



# MSTL fit

In [17]:
models = [MSTL(season_length=[7, 365], trend_forecaster=AutoARIMA())]
sf = StatsForecast(models=models, freq='D')
sf.fit(df=df_8)



StatsForecast(models=[MSTL])

In [19]:
result=sf.fitted_[0,0].model_
result

,data,trend,seasonal7,seasonal365,remainder
0,106.0,79.182600,8.627888,8.608943,9.580569
1,91.0,79.260566,0.323397,8.582155,2.833882
2,115.0,79.338526,3.000392,6.166305,26.494777
3,104.0,79.416481,2.298993,-0.526880,22.811406
4,107.0,79.494430,0.018244,13.170299,14.317027
...,...,...,...,...,...
938,75.0,129.980401,-46.361924,-3.402739,-5.215738
939,112.0,130.012639,0.856476,-4.103967,-14.765148
940,137.0,130.044884,15.825538,4.612189,-13.482612
941,156.0,130.077137,31.203824,-14.338125,9.057164


# MSTL predict_in_sample 

In [58]:
y = df_8['y'].values  
model = AutoARIMA()
fitted_model = model.fit(y)
in_sample_predictions = fitted_model.predict_in_sample(level=[80, 95])

In [63]:
in_sample_predictions

{'fitted': array([105.89400009, 102.78408916,  98.14654127, 111.38843886,
        103.68862502, 102.87968984,  89.29183163,  87.19482119,
         91.82303613,  93.91171965,  83.72904921,  78.70959661,
         71.71621169,  60.58352919,  66.15463957,  72.47489668,
         74.00642559,  73.81407514,  72.56578342,  71.47962617,
         64.75543464,  72.02746926,  79.78403663,  76.88503624,
         85.60757595,  74.96953036,  77.26045841,  79.13332374,
         68.19381106,  93.01807911,  87.84412268,  90.98674719,
         87.53610376,  76.19665745,  81.35388666,  81.83052091,
         96.05180439,  94.25882828,  79.96858334,  85.75599512,
         77.78936895,  91.75866166,  99.33117835, 104.67586125,
         78.65082324,  81.9850635 ,  91.21912446,  95.06918209,
         88.51583397,  94.80403873,  73.9832274 ,  77.74983998,
         95.21198264, 102.28959757,  95.45503678,  89.94583776,
         90.70884925,  92.8468668 ,  98.2263567 ,  94.6287354 ,
         94.37833792,  83.2324

In [ ]:
# Verificar as chaves do dicionário
print(in_sample_predictions.keys())

dict_keys(['fitted', 'fitted-lo-95', 'fitted-lo-80', 'fitted-hi-80', 'fitted-hi-95'])


In [69]:
in_sample_predictions_df = pd.DataFrame({
    'y': df_8['y'], 
    'fitted': in_sample_predictions['fitted'],
    'fitted-lo-95': in_sample_predictions['fitted-lo-95'],
    'fitted-hi-95': in_sample_predictions['fitted-hi-95']
})

# Identificar anomalias fora do intervalo de confiança de 95%
anomalies = in_sample_predictions_df[~in_sample_predictions_df['y'].between(in_sample_predictions_df['fitted-lo-95'], in_sample_predictions_df['fitted-hi-95'])]

print(anomalies)

       y      fitted  fitted-lo-95  fitted-hi-95
95   177  104.514487     43.554964    165.474010
142   39  112.469658     51.510135    173.429181
146  276   47.350984    -13.608539    108.310507
147  389  158.800100     97.840577    219.759623
148   50  198.663884    137.704361    259.623407
153   64  176.909019    115.949496    237.868542
372   14  136.342968     75.383445    197.302491
408  233  138.867375     77.907852    199.826898
409   98  181.777131    120.817608    242.736654
419   74  141.516267     80.556744    202.475790
435   84  152.246899     91.287376    213.206422
444   90  174.672671    113.713148    235.632194
450  281  135.957928     74.998405    196.917451
456   94  164.988389    104.028866    225.947912
465  183  116.202988     55.243465    177.162511
487   70  142.953279     81.993756    203.912802
498  143   76.430223     15.470700    137.389746
519  121   59.612272     -1.347251    120.571795
523  187  112.775461     51.815938    173.734984
532  163   71.143585